#Creando Funcion sliding_window_view

In [ ]:
import numpy as np
from numpy.core.numeric import normalize_axis_tuple
from numpy.core.overrides import array_function_dispatch, set_module

__all__ = ['broadcast_to', 'broadcast_arrays', 'broadcast_shapes']


class DummyArray:
    """Dummy object that just exists to hang __array_interface__ dictionaries
    and possibly keep alive a reference to a base array.
    """

    def __init__(self, interface, base=None):
        self.__array_interface__ = interface
        self.base = base


def _maybe_view_as_subclass(original_array, new_array):
    if type(original_array) is not type(new_array):
        # if input was an ndarray subclass and subclasses were OK,
        # then view the result as that subclass.
        new_array = new_array.view(type=type(original_array))
        # Since we have done something akin to a view from original_array, we
        # should let the subclass finalize (if it has it implemented, i.e., is
        # not None).
        if new_array.__array_finalize__:
            new_array.__array_finalize__(original_array)
    return new_array


def as_strided(x, shape=None, strides=None, subok=False, writeable=True):
    """
    Create a view into the array with the given shape and strides.
    .. warning:: This function has to be used with extreme care, see notes.
    Parameters
    ----------
    x : ndarray
        Array to create a new.
    shape : sequence of int, optional
        The shape of the new array. Defaults to ``x.shape``.
    strides : sequence of int, optional
        The strides of the new array. Defaults to ``x.strides``.
    subok : bool, optional
        .. versionadded:: 1.10
        If True, subclasses are preserved.
    writeable : bool, optional
        .. versionadded:: 1.12
        If set to False, the returned array will always be readonly.
        Otherwise it will be writable if the original array was. It
        is advisable to set this to False if possible (see Notes).
    Returns
    -------
    view : ndarray
    See also
    --------
    broadcast_to : broadcast an array to a given shape.
    reshape : reshape an array.
    lib.stride_tricks.sliding_window_view :
        userfriendly and safe function for the creation of sliding window views.
    Notes
    -----
    ``as_strided`` creates a view into the array given the exact strides
    and shape. This means it manipulates the internal data structure of
    ndarray and, if done incorrectly, the array elements can point to
    invalid memory and can corrupt results or crash your program.
    It is advisable to always use the original ``x.strides`` when
    calculating new strides to avoid reliance on a contiguous memory
    layout.
    Furthermore, arrays created with this function often contain self
    overlapping memory, so that two elements are identical.
    Vectorized write operations on such arrays will typically be
    unpredictable. They may even give different results for small, large,
    or transposed arrays.
    Since writing to these arrays has to be tested and done with great
    care, you may want to use ``writeable=False`` to avoid accidental write
    operations.
    For these reasons it is advisable to avoid ``as_strided`` when
    possible.
    """
    # first convert input to array, possibly keeping subclass
    x = np.array(x, copy=False, subok=subok)
    interface = dict(x.__array_interface__)
    if shape is not None:
        interface['shape'] = tuple(shape)
    if strides is not None:
        interface['strides'] = tuple(strides)

    array = np.asarray(DummyArray(interface, base=x))
    # The route via `__interface__` does not preserve structured
    # dtypes. Since dtype should remain unchanged, we set it explicitly.
    array.dtype = x.dtype

    view = _maybe_view_as_subclass(x, array)

    if view.flags.writeable and not writeable:
        view.flags.writeable = False

    return view


def _sliding_window_view_dispatcher(x, window_shape, axis=None, *,
                                    subok=None, writeable=None):
    return (x,)


@array_function_dispatch(_sliding_window_view_dispatcher)
def sliding_window_view(x, window_shape, axis=None, *,
                        subok=False, writeable=False):
    """
    Create a sliding window view into the array with the given window shape.
    Also known as rolling or moving window, the window slides across all
    dimensions of the array and extracts subsets of the array at all window
    positions.
    
    .. versionadded:: 1.20.0
    Parameters
    ----------
    x : array_like
        Array to create the sliding window view from.
    window_shape : int or tuple of int
        Size of window over each axis that takes part in the sliding window.
        If `axis` is not present, must have same length as the number of input
        array dimensions. Single integers `i` are treated as if they were the
        tuple `(i,)`.
    axis : int or tuple of int, optional
        Axis or axes along which the sliding window is applied.
        By default, the sliding window is applied to all axes and
        `window_shape[i]` will refer to axis `i` of `x`.
        If `axis` is given as a `tuple of int`, `window_shape[i]` will refer to
        the axis `axis[i]` of `x`.
        Single integers `i` are treated as if they were the tuple `(i,)`.
    subok : bool, optional
        If True, sub-classes will be passed-through, otherwise the returned
        array will be forced to be a base-class array (default).
    writeable : bool, optional
        When true, allow writing to the returned view. The default is false,
        as this should be used with caution: the returned view contains the
        same memory location multiple times, so writing to one location will
        cause others to change.
    Returns
    -------
    view : ndarray
        Sliding window view of the array. The sliding window dimensions are
        inserted at the end, and the original dimensions are trimmed as
        required by the size of the sliding window.
        That is, ``view.shape = x_shape_trimmed + window_shape``, where
        ``x_shape_trimmed`` is ``x.shape`` with every entry reduced by one less
        than the corresponding window size.
    See Also
    --------
    lib.stride_tricks.as_strided: A lower-level and less safe routine for
        creating arbitrary views from custom shape and strides.
    broadcast_to: broadcast an array to a given shape.
    Notes
    -----
    For many applications using a sliding window view can be convenient, but
    potentially very slow. Often specialized solutions exist, for example:
    - `scipy.signal.fftconvolve`
    - filtering functions in `scipy.ndimage`
    - moving window functions provided by
      `bottleneck <https://github.com/pydata/bottleneck>`_.
    As a rough estimate, a sliding window approach with an input size of `N`
    and a window size of `W` will scale as `O(N*W)` where frequently a special
    algorithm can achieve `O(N)`. That means that the sliding window variant
    for a window size of 100 can be a 100 times slower than a more specialized
    version.
    Nevertheless, for small window sizes, when no custom algorithm exists, or
    as a prototyping and developing tool, this function can be a good solution.
    Examples
    --------
    >>> x = np.arange(6)
    >>> x.shape
    (6,)
    >>> v = sliding_window_view(x, 3)
    >>> v.shape
    (4, 3)
    >>> v
    array([[0, 1, 2],
           [1, 2, 3],
           [2, 3, 4],
           [3, 4, 5]])
    This also works in more dimensions, e.g.
    >>> i, j = np.ogrid[:3, :4]
    >>> x = 10*i + j
    >>> x.shape
    (3, 4)
    >>> x
    array([[ 0,  1,  2,  3],
           [10, 11, 12, 13],
           [20, 21, 22, 23]])
    >>> shape = (2,2)
    >>> v = sliding_window_view(x, shape)
    >>> v.shape
    (2, 3, 2, 2)
    >>> v
    array([[[[ 0,  1],
             [10, 11]],
            [[ 1,  2],
             [11, 12]],
            [[ 2,  3],
             [12, 13]]],
           [[[10, 11],
             [20, 21]],
            [[11, 12],
             [21, 22]],
            [[12, 13],
             [22, 23]]]])
    The axis can be specified explicitly:
    >>> v = sliding_window_view(x, 3, 0)
    >>> v.shape
    (1, 4, 3)
    >>> v
    array([[[ 0, 10, 20],
            [ 1, 11, 21],
            [ 2, 12, 22],
            [ 3, 13, 23]]])
    The same axis can be used several times. In that case, every use reduces
    the corresponding original dimension:
    >>> v = sliding_window_view(x, (2, 3), (1, 1))
    >>> v.shape
    (3, 1, 2, 3)
    >>> v
    array([[[[ 0,  1,  2],
             [ 1,  2,  3]]],
           [[[10, 11, 12],
             [11, 12, 13]]],
           [[[20, 21, 22],
             [21, 22, 23]]]])
    Combining with stepped slicing (`::step`), this can be used to take sliding
    views which skip elements:
    >>> x = np.arange(7)
    >>> sliding_window_view(x, 5)[:, ::2]
    array([[0, 2, 4],
           [1, 3, 5],
           [2, 4, 6]])
    or views which move by multiple elements
    >>> x = np.arange(7)
    >>> sliding_window_view(x, 3)[::2, :]
    array([[0, 1, 2],
           [2, 3, 4],
           [4, 5, 6]])
    A common application of `sliding_window_view` is the calculation of running
    statistics. The simplest example is the
    `moving average <https://en.wikipedia.org/wiki/Moving_average>`_:
    >>> x = np.arange(6)
    >>> x.shape
    (6,)
    >>> v = sliding_window_view(x, 3)
    >>> v.shape
    (4, 3)
    >>> v
    array([[0, 1, 2],
           [1, 2, 3],
           [2, 3, 4],
           [3, 4, 5]])
    >>> moving_average = v.mean(axis=-1)
    >>> moving_average
    array([1., 2., 3., 4.])
    Note that a sliding window approach is often **not** optimal (see Notes).
    """
    window_shape = (tuple(window_shape)
                    if np.iterable(window_shape)
                    else (window_shape,))
    # first convert input to array, possibly keeping subclass
    x = np.array(x, copy=False, subok=subok)

    window_shape_array = np.array(window_shape)
    if np.any(window_shape_array < 0):
        raise ValueError('`window_shape` cannot contain negative values')

    if axis is None:
        axis = tuple(range(x.ndim))
        if len(window_shape) != len(axis):
            raise ValueError(f'Since axis is `None`, must provide '
                             f'window_shape for all dimensions of `x`; '
                             f'got {len(window_shape)} window_shape elements '
                             f'and `x.ndim` is {x.ndim}.')
    else:
        axis = normalize_axis_tuple(axis, x.ndim, allow_duplicate=True)
        if len(window_shape) != len(axis):
            raise ValueError(f'Must provide matching length window_shape and '
                             f'axis; got {len(window_shape)} window_shape '
                             f'elements and {len(axis)} axes elements.')

    out_strides = x.strides + tuple(x.strides[ax] for ax in axis)

    # note: same axis can be windowed repeatedly
    x_shape_trimmed = list(x.shape)
    for ax, dim in zip(axis, window_shape):
        if x_shape_trimmed[ax] < dim:
            raise ValueError(
                'window shape cannot be larger than input array shape')
        x_shape_trimmed[ax] -= dim - 1
    out_shape = tuple(x_shape_trimmed) + window_shape
    return as_strided(x, strides=out_strides, shape=out_shape,
                      subok=subok, writeable=writeable)


def _broadcast_to(array, shape, subok, readonly):
    shape = tuple(shape) if np.iterable(shape) else (shape,)
    array = np.array(array, copy=False, subok=subok)
    if not shape and array.shape:
        raise ValueError('cannot broadcast a non-scalar to a scalar array')
    if any(size < 0 for size in shape):
        raise ValueError('all elements of broadcast shape must be non-'
                         'negative')
    extras = []
    it = np.nditer(
        (array,), flags=['multi_index', 'refs_ok', 'zerosize_ok'] + extras,
        op_flags=['readonly'], itershape=shape, order='C')
    with it:
        # never really has writebackifcopy semantics
        broadcast = it.itviews[0]
    result = _maybe_view_as_subclass(array, broadcast)
    # In a future version this will go away
    if not readonly and array.flags._writeable_no_warn:
        result.flags.writeable = True
        result.flags._warn_on_write = True
    return result


def _broadcast_to_dispatcher(array, shape, subok=None):
    return (array,)


@array_function_dispatch(_broadcast_to_dispatcher, module='numpy')
def broadcast_to(array, shape, subok=False):
    """Broadcast an array to a new shape.
    Parameters
    ----------
    array : array_like
        The array to broadcast.
    shape : tuple or int
        The shape of the desired array. A single integer ``i`` is interpreted
        as ``(i,)``.
    subok : bool, optional
        If True, then sub-classes will be passed-through, otherwise
        the returned array will be forced to be a base-class array (default).
    Returns
    -------
    broadcast : array
        A readonly view on the original array with the given shape. It is
        typically not contiguous. Furthermore, more than one element of a
        broadcasted array may refer to a single memory location.
    Raises
    ------
    ValueError
        If the array is not compatible with the new shape according to NumPy's
        broadcasting rules.
    See Also
    --------
    broadcast
    broadcast_arrays
    broadcast_shapes
    Notes
    -----
    .. versionadded:: 1.10.0
    Examples
    --------
    >>> x = np.array([1, 2, 3])
    >>> np.broadcast_to(x, (3, 3))
    array([[1, 2, 3],
           [1, 2, 3],
           [1, 2, 3]])
    """
    return _broadcast_to(array, shape, subok=subok, readonly=True)


def _broadcast_shape(*args):
    """Returns the shape of the arrays that would result from broadcasting the
    supplied arrays against each other.
    """
    # use the old-iterator because np.nditer does not handle size 0 arrays
    # consistently
    b = np.broadcast(*args[:32])
    # unfortunately, it cannot handle 32 or more arguments directly
    for pos in range(32, len(args), 31):
        # ironically, np.broadcast does not properly handle np.broadcast
        # objects (it treats them as scalars)
        # use broadcasting to avoid allocating the full array
        b = broadcast_to(0, b.shape)
        b = np.broadcast(b, *args[pos:(pos + 31)])
    return b.shape


@set_module('numpy')
def broadcast_shapes(*args):
    """
    Broadcast the input shapes into a single shape.
    :ref:`Learn more about broadcasting here <basics.broadcasting>`.
    .. versionadded:: 1.20.0
    Parameters
    ----------
    `*args` : tuples of ints, or ints
        The shapes to be broadcast against each other.
    Returns
    -------
    tuple
        Broadcasted shape.
    Raises
    ------
    ValueError
        If the shapes are not compatible and cannot be broadcast according
        to NumPy's broadcasting rules.
    See Also
    --------
    broadcast
    broadcast_arrays
    broadcast_to
    Examples
    --------
    >>> np.broadcast_shapes((1, 2), (3, 1), (3, 2))
    (3, 2)
    >>> np.broadcast_shapes((6, 7), (5, 6, 1), (7,), (5, 1, 7))
    (5, 6, 7)
    """
    arrays = [np.empty(x, dtype=[]) for x in args]
    return _broadcast_shape(*arrays)


def _broadcast_arrays_dispatcher(*args, subok=None):
    return args


@array_function_dispatch(_broadcast_arrays_dispatcher, module='numpy')
def broadcast_arrays(*args, subok=False):
    """
    Broadcast any number of arrays against each other.
    Parameters
    ----------
    `*args` : array_likes
        The arrays to broadcast.
    subok : bool, optional
        If True, then sub-classes will be passed-through, otherwise
        the returned arrays will be forced to be a base-class array (default).
    Returns
    -------
    broadcasted : list of arrays
        These arrays are views on the original arrays.  They are typically
        not contiguous.  Furthermore, more than one element of a
        broadcasted array may refer to a single memory location. If you need
        to write to the arrays, make copies first. While you can set the
        ``writable`` flag True, writing to a single output value may end up
        changing more than one location in the output array.
        .. deprecated:: 1.17
            The output is currently marked so that if written to, a deprecation
            warning will be emitted. A future version will set the
            ``writable`` flag False so writing to it will raise an error.
    See Also
    --------
    broadcast
    broadcast_to
    broadcast_shapes
    Examples
    --------
    >>> x = np.array([[1,2,3]])
    >>> y = np.array([[4],[5]])
    >>> np.broadcast_arrays(x, y)
    [array([[1, 2, 3],
           [1, 2, 3]]), array([[4, 4, 4],
           [5, 5, 5]])]
    Here is a useful idiom for getting contiguous copies instead of
    non-contiguous views.
    >>> [np.array(a) for a in np.broadcast_arrays(x, y)]
    [array([[1, 2, 3],
           [1, 2, 3]]), array([[4, 4, 4],
           [5, 5, 5]])]
    """
    # nditer is not used here to avoid the limit of 32 arrays.
    # Otherwise, something like the following one-liner would suffice:
    # return np.nditer(args, flags=['multi_index', 'zerosize_ok'],
    #                  order='C').itviews

    args = [np.array(_m, copy=False, subok=subok) for _m in args]

    shape = _broadcast_shape(*args)

    if all(array.shape == shape for array in args):
        # Common case where nothing needs to be broadcasted.
        return args

    return [_broadcast_to(array, shape, subok=subok, readonly=False)
            for array in args]

#Seguir

In [ ]:
#@title Importando librerias
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from numpy import save

In [ ]:
#@title Montando Drive
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive/Semestre 8/IA-2/Proyecto IA')
print(os.getcwd())

Mounted at /content/drive
/content/drive/My Drive/Semestre 8/IA-2/Proyecto IA


#Limpiando data

No es necesario correr, ya quedo limpia

In [ ]:
#@title Leyendo datasets
path='./Datasets/Full History/'
markets = ["FB","AMZN","NFLX","AAPL","MSFT","GOOGL","TWTR","UBER","NVDA","BABA"] 

li = []

for market in markets:
    df = pd.read_csv(path + market +".csv", index_col=None, header=0)
    df["Symbol"] = market
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
#@title Modificando columnas y partiendo train/test
data_clean_train = []
data_clean_test = []

for d in li:
  d.dropna(inplace=True)
  d['time'] = pd.to_datetime(d['time'],format='%Y-%m-%d %H:%M:%S')
  d.high=d.high.astype('float32')
  d.low=d.low.astype('float32')
  d.open=d.open.astype('float32')
  d.close=d.close.astype('float32')
  d.volume=d.volume.astype('float32')
  d['Volatility']= (d.high - d.low)*100/d.open
  d['Displacement']= (d.close - d.open)*100/df.open
  d.sort_values("time",ignore_index=True,inplace=True)

  train = d[:int(d.shape[0]*0.8)-1].copy()
  test= d[int(d.shape[0]*0.8):].copy()

  data_clean_train.append(train)
  data_clean_test.append(test)

In [ ]:
#@title Escalando datasets individualmente

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

for i in range(0,len(data_clean_train)):
  cols = data_clean_train[i].columns
  #Creando escaladores
  scal1 = StandardScaler()
  scal2 = MinMaxScaler()
  #Ajustando y transformando Train
  data_clean_train[i][['open_std_sc','high_std_sc','low_std_sc','close_std_sc','volume_std_sc','Volatility_std_sc','Displacement_std_sc']] = scal1.fit_transform(data_clean_train[i][['open','high','low','close','volume','Volatility','Displacement']])
  data_clean_train[i][['open_MinMax_sc','high_MinMax_sc','low_MinMax_sc','close_MinMax_sc','volume_MinMax_sc','Volatility_MinMax_sc','Displacement_MinMax_sc']] = scal2.fit_transform(data_clean_train[i][['open','high','low','close','volume','Volatility','Displacement']])
 
 #Transformando data de test
  data_clean_test[i][['open_std_sc','high_std_sc','low_std_sc','close_std_sc','volume_std_sc','Volatility_std_sc','Displacement_std_sc']] = scal1.transform(data_clean_test[i][['open','high','low','close','volume','Volatility','Displacement']])
  data_clean_test[i][['open_MinMax_sc','high_MinMax_sc','low_MinMax_sc','close_MinMax_sc','volume_MinMax_sc','Volatility_MinMax_sc','Displacement_MinMax_sc']] = scal2.transform(data_clean_test[i][['open','high','low','close','volume','Volatility','Displacement']])


In [ ]:
#@title Salvando train/test
for i in range(0,len(data_clean_train)):
  market = data_clean_train[i].Symbol[0]
  data_clean_train[i].to_csv("Datasets/Train/"+market+".csv",index=False)
  data_clean_test[i].to_csv("Datasets/Test/"+market+".csv",index=False)
  print(market+".csv Saved succesfully")

FB.csv Saved succesfully
AMZN.csv Saved succesfully
NFLX.csv Saved succesfully
AAPL.csv Saved succesfully
MSFT.csv Saved succesfully
GOOGL.csv Saved succesfully
TWTR.csv Saved succesfully
UBER.csv Saved succesfully
NVDA.csv Saved succesfully
BABA.csv Saved succesfully


#Unificando Datasets
Queda faltando guardar la data

##CNN part
Datos ya guardados

In [ ]:
path1='./Datasets/Train/'
path2='./Datasets/Test/'
markets = ["FB","AMZN","NFLX","AAPL","MSFT","GOOGL","TWTR","UBER","NVDA","BABA"] 

li1 = []
li2 = []

for market in markets:
    df1 = pd.read_csv(path1 + market +".csv", index_col=None, header=0)
    df1["Symbol"] = market

    df2 = pd.read_csv(path2 + market +".csv", index_col=None, header=0)
    df2["Symbol"] = market

    li1.append(df1)
    li2.append(df2)

#df_train = pd.concat(li1, axis=0, ignore_index=True)
#df_test = pd.concat(li2, axis=0, ignore_index=True)

In [ ]:
cols = ['open', 'high', 'low', 'close', 'volume',
       'Volatility', 'Displacement', 'open_std_sc', 'high_std_sc',
       'low_std_sc', 'close_std_sc', 'volume_std_sc', 'Volatility_std_sc',
       'Displacement_std_sc', 'open_MinMax_sc', 'high_MinMax_sc',
       'low_MinMax_sc', 'close_MinMax_sc', 'volume_MinMax_sc',
       'Volatility_MinMax_sc', 'Displacement_MinMax_sc']

In [ ]:
window_size = 30
train_wind_list = []
test_wind_list = []
for i in range(0,len(li1)):
  slices1=[]
  slices2=[]
  for col in cols:
    #Creando las ventanas
    sli1 = sliding_window_view(li1[i][col].values,window_size)
    slices1.append(np.expand_dims(sli1,axis=2))

    sli2 = sliding_window_view(li2[i][col].values,window_size)
    slices2.append(np.expand_dims(sli2,axis=2))

  #Concatenando los datasets ventaneados
  windowed_df1 = np.dstack(slices1)[:-1,:,:]
  train_wind_list.append(windowed_df1)

  windowed_df2 = np.dstack(slices2)[:-1,:,:]
  test_wind_list.append(windowed_df2)
  

In [ ]:
train_cnn = np.vstack(train_wind_list)
test_cnn = np.vstack(test_wind_list)
print(train_cnn.shape)
print(test_cnn.shape)

(130019, 30, 21)
(32289, 30, 21)


In [ ]:
#Obteniendo las variables objetivo
train_objetivo = np.hstack([d["high"].values[window_size:] for d in li1])
test_objetivo = np.hstack([d["high"].values[window_size:] for d in li2])
print(train_objetivo.shape)
print(test_objetivo.shape)

(130019,)
(32289,)


In [ ]:
#@title Salvando data
save('Datasets/Train/x_train_cnn.npy', train_cnn)
save('Datasets/Test/x_test_cnn.npy', test_cnn)
save('Datasets/Train/y_train_cnn.npy', train_objetivo)
save('Datasets/Test/y_test_cnn.npy', test_objetivo)

In [ ]:
v1=li1[2]["high_std_sc"][30:].values
v1.shape

(8845,)

In [ ]:
v2=train_objetivo[12801+9482:12801+9482+8845]
v2.shape

(8845,)

In [ ]:
(v1==v2).all()

True

#Pruebas tensorflow

In [ ]:
window_size=30
batch_size=21
tens = tf.data.Dataset.from_tensor_slices(df_train[cols])
tens = tens.window(window_size + 1, shift=1, drop_remainder=True)
tens = tens.flat_map(lambda window: window.batch(window_size + 1))
tens = tens.map(lambda window: (window[:-1], window[-1]))
tens = tens.batch(batch_size).prefetch(1)

In [ ]:
for i, (x,y) in enumerate(tens):
  if i == 1:
    print('Number of samples in each batch: ', x.numpy().shape[0])
    print('Number of labels in each batch: ', y.numpy().shape[0])
    print('Number of features of each sample (temperature days): ', x.numpy().shape[1])
    print('---------------------')
    print('X_1 = \n', x.numpy()[1])
    print('X_2 = \n', x.numpy()[2])
    print('X_3 = \n', x.numpy()[3])
    print('X_3 = \n', x.numpy()[20])
    print('\n')
    print('Y_1 = ', y.numpy()[1])

Number of samples in each batch:  21
Number of labels in each batch:  21
Number of features of each sample (temperature days):  30
---------------------
X_1 = 
 [[ 3.52100000e+02  3.52100000e+02  3.52000000e+02  3.52000000e+02
   1.62400000e+03  2.84027550e-02 -2.84027559e-02  5.70426265e-01
   5.56730146e-01  5.78836887e-01  5.65231319e-01 -6.31401363e-01
  -5.64249251e-01 -1.74603849e-01  6.17998408e-01  6.16264029e-01
   6.21044511e-01  6.17088462e-01  4.41440614e-04  3.62954697e-03
   7.04723315e-01]
 [ 3.52090000e+02  3.52090000e+02  3.51670000e+02  3.51990000e+02
   2.22900000e+03  1.19282830e-01 -2.84035626e-02  5.69894169e-01
   5.56197226e-01  5.61328650e-01  5.64699305e-01 -6.28274560e-01
  -1.84431897e-01 -1.74609005e-01  6.17879102e-01  6.16144851e-01
   6.17140961e-01  6.16969262e-01  6.16684428e-04  1.52429798e-02
   7.04723230e-01]
 [ 3.52030000e+02  3.52030000e+02  3.52010000e+02  3.52020000e+02
   1.25600000e+03  5.67821300e-03 -2.84344108e-03  5.66704833e-01
   5.5300